In [1]:
#meta 10/25/2020 myStudy Ch10. NLP - replicate with a dataset Yelp reviews

#$note: to follow-up with a cleaner version

#focused on
# data load and data prep
# make it work with my dataset for study group time on 10/27/2020

# to follow-up: fastai.collab parts only

In [2]:
#hide
#!pip install -Uqq fastbook
#import fastbook
#fastbook.setup_book()

In [1]:
#hide
from fastbook import *
from IPython.display import display,HTML

# NLP Deep Dive: RNNs
#### my Shallow Dive
The objective is to build a language model with my Yelp reviews dataset and train a sentiment classifier.

## 0. Load Data
The Dataset: 
NOT Swift Keyboard (sample) - blogs, news and twitter entries.   
Way smaller dataset.  Full dataset is much bigger and can experiment once figure out this small one. 

take 2. Yelp reviews, yelp_reviews.zip

In [2]:
from fastai.text.all import *
path = URLs.path('yelp_reviews')
path

Path('/home/jupyter/.fastai/archive/yelp_reviews')

In [3]:
path.ls()

(#2) [Path('/home/jupyter/.fastai/archive/yelp_reviews/train'),Path('/home/jupyter/.fastai/archive/yelp_reviews/models')]

In [6]:
#Path.BASE_PATH = path

In [4]:
files = get_text_files(path, folders = ['train'])
files

(#560000) [Path('/home/jupyter/.fastai/archive/yelp_reviews/train/negative/374288.txt'),Path('/home/jupyter/.fastai/archive/yelp_reviews/train/negative/62916.txt'),Path('/home/jupyter/.fastai/archive/yelp_reviews/train/negative/395724.txt'),Path('/home/jupyter/.fastai/archive/yelp_reviews/train/negative/401257.txt'),Path('/home/jupyter/.fastai/archive/yelp_reviews/train/negative/381201.txt'),Path('/home/jupyter/.fastai/archive/yelp_reviews/train/negative/192865.txt'),Path('/home/jupyter/.fastai/archive/yelp_reviews/train/negative/411821.txt'),Path('/home/jupyter/.fastai/archive/yelp_reviews/train/negative/373852.txt'),Path('/home/jupyter/.fastai/archive/yelp_reviews/train/negative/379305.txt'),Path('/home/jupyter/.fastai/archive/yelp_reviews/train/negative/4123.txt')...]

In [5]:
txt = files[0].open().read(); txt[:2000]

"Cab #785. Took cab from hotel to airport. The hotel told me to make sure to ask to go through paradise. But the cab initially wanted to take the highway (much more miles and not faster - I had google maps up). While following my instructions initially the driver decided to go his own way despite my repeated request to go my way. He then proceeded to tell me that I could drive if i wanted and that his job was to take me to the airport the way he wanted. He then insisted again on how his way was better (it was slightly longer). Also they dont tell you until when you are about to click on okay that there is a $3 charge to pay with credit card. Very bad experience. There was no complain number and it was hard to see the driver's ID."

## 1. Text Preprocessing

### Word Tokenization with fastai
The default English word tokenizer for fastai uses a library called *spaCy*. It has a sophisticated rules engine with special rules for URLs, individual special English words, and much more. Rather than directly using `SpacyTokenizer`, however, we'll use `WordTokenizer`, since that will always point to fastai's current default word tokenizer (which may not necessarily be spaCy, depending when you're reading this).

Let's try it out. We'll use fastai's `coll_repr(collection, n)` function to display the results. This displays the first *`n`* items of *`collection`*, along with the full size—it's what `L` uses by default. Note that fastai's tokenizers take a collection of documents to tokenize, so we have to wrap `txt` in a list

In [6]:
spacy = WordTokenizer() #class fastai.text.core.SpacyTokenizer
#spacy([txt]) #class generator
toks = first(spacy([txt])) #class fastcore.foundation.L
print(coll_repr(toks, 30)) #class str

(#165) ['Cab','#','785','.','Took','cab','from','hotel','to','airport','.','The','hotel','told','me','to','make','sure','to','ask','to','go','through','paradise','.','But','the','cab','initially','wanted'...]


In [10]:
#first(spacy(['The U.S. dollar $1 is $1.00.']))

fastai then adds some additional functionality to the tokenization process with the `Tokenizer` class:

In [7]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#178) ['xxbos','xxmaj','cab','#','785','.','xxmaj','took','cab','from','hotel','to','airport','.','xxmaj','the','hotel','told','me','to','make','sure','to','ask','to','go','through','paradise','.','xxmaj','but'...]


Here are some of the main special tokens you'll see:

- `xxbos`:: Indicates the beginning of a text (here, a review)
- `xxmaj`:: Indicates the next word begins with a capital (since we lowercased everything)
- `xxunk`:: Indicates the next word is unknown

To see the rules that were used, you can check the default rules

In [8]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [13]:
#coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

### Subword Tokenization
This proceeds in two steps:

1. Analyze a corpus of documents to find the most commonly occurring groups of letters. These become the vocab.
2. Tokenize the corpus using this vocab of *subword units*.

In [14]:
txts = L(o.open().read() for o in files[:2000])

In [15]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [16]:
subword(1000)

'▁Ca b ▁ # 7 8 5 . ▁T ook ▁ca b ▁from ▁hotel ▁to ▁airport . ▁The ▁hotel ▁told ▁me ▁to ▁make ▁sure ▁to ▁ask ▁to ▁go ▁through ▁pa ra d ise . ▁But ▁the ▁ca b ▁in it'

In [17]:
subword(200)

'▁ C a b ▁ # 7 8 5 . ▁ T o o k ▁c a b ▁f ro m ▁ho t e l ▁to ▁a ir p or t . ▁The ▁ho t e l ▁to l d'

In [18]:
#longer vocab basically every word becomes a token
subword(10000)

'▁Cab ▁#7 85 . ▁Too k ▁cab ▁from ▁hotel ▁to ▁airport . ▁The ▁hotel ▁told ▁me ▁to ▁make ▁sure ▁to ▁ask ▁to ▁go ▁through ▁paradise . ▁But ▁the ▁cab ▁initially ▁wanted ▁to ▁take ▁the ▁highway ▁( m uch ▁more ▁miles'

### Numericalization with fastai

In [19]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#178) ['xxbos','xxmaj','cab','#','785','.','xxmaj','took','cab','from','hotel','to','airport','.','xxmaj','the','hotel','told','me','to','make','sure','to','ask','to','go','through','paradise','.','xxmaj','but'...]


In [20]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#178) ['xxbos','xxmaj','cab','#','785','.','xxmaj','took','cab','from'...]

In [21]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1440) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','.','the',',','i','to','and','a','was','it','of','for'...]"

Our special rules tokens appear first, and then every word appears once, in frequency order. The defaults to `Numericalize` are `min_freq=3,max_vocab=60000`. `max_vocab=60000` results in fastai replacing all words other than the most common 60,000 with a special *unknown word* token, `xxunk`. This is useful to avoid having an overly large embedding matrix, since that can slow down training and use up too much memory, and can also mean that there isn't enough data to train useful representations for rare words. However, this last issue is better handled by setting `min_freq`; the default `min_freq=3` means that any word appearing less than three times is replaced with `xxunk`.

fastai can also numericalize your dataset using a vocab that you provide, by passing a list of words as the `vocab` parameter.

Once we've created our `Numericalize` object, we can use it as if it were a function

In [22]:
nums = num(toks)[:20]; nums

tensor([   2,    8, 1094,  352,    0,    9,    8,  156, 1094,   72,  233,   13,  561,    9,    8,   10,  233,  119,   41,   13])

In [23]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj cab # xxunk . xxmaj took cab from hotel to airport . xxmaj the hotel told me to'

Now that we have numbers, we need to put them in batches for our model.

### Putting Our Texts into Batches for a Language Model

We want our language model to read text in order, so that it can efficiently predict what the next word is. This means that each new batch should begin precisely where the previous one left off.

Example: We have text t. The tokenization process will add special tokens and deal with punctuation to return text t tokenized.

We now have 90 tokens, separated by spaces. Let's say we want a batch size of 6. We need to break this text into 6 contiguous parts of length 15

In [24]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


$ac minibatches are formed differently  

We need to divide this array more finely into subarrays of a fixed sequence length. It is important to maintain order within and across these subarrays, because we will use a model that maintains a state so that it remembers what it read previously when predicting what comes next. 

Going back to our previous example with 6 batches of length 15, if we chose a sequence length of 5, that would mean we first feed the following array

In [25]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [26]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [27]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


Going back to our movie reviews dataset, the first step is to transform the individual texts into a stream by concatenating them together. As with images, it's best to randomize the order of the inputs, so at the beginning of each epoch we will shuffle the entries to make a new stream (we shuffle the order of the documents, not the order of the words inside them, or the texts would not make sense anymore!).

We then cut this stream into a certain number of batches (which is our *batch size*). For instance, if the stream has 50,000 tokens and we set a batch size of 10, this will give us 10 mini-streams of 5,000 tokens. What is important is that we preserve the order of the tokens (so from 1 to 5,000 for the first mini-stream, then from 5,001 to 10,000...), because we want the model to read continuous rows of text (as in the preceding example). An `xxbos` token is added at the start of each during preprocessing, so that the model knows when it reads the stream when a new entry is beginning.

So to recap, at every epoch we shuffle our collection of documents and concatenate them into a stream of tokens. We then cut that stream into a batch of fixed-size consecutive mini-streams. Our model will then read the mini-streams in order, and thanks to an inner state, it will produce the same activation whatever sequence length we picked.

This is all done behind the scenes by the fastai library when we create an `LMDataLoader`. We do this by first applying our `Numericalize` object to the tokenized texts

In [ ]:
nums200 = toks200.map(num)

In [ ]:
dl = LMDataLoader(nums200)

In [ ]:
x,y = first(dl)
x.shape,y.shape

In [ ]:
' '.join(num.vocab[o] for o in x[0][:20])

In [32]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj cab # xxunk . xxmaj took cab from hotel to airport . xxmaj the hotel told me to make'

This concludes all the preprocessing steps we need to apply to our data. We are now ready to train our text classifier.

## Training a Text Classifier
There are two steps to training a state-of-the-art text classifier using transfer learning:  
1 - we need to fine-tune our language model pretrained on Wikipedia to the corpus of IMDb reviews, and  
2 - we can use that model to train a classifier.

As usual, let's start with assembling our data.

### Language Model Using DataBlock
> fast.ai DataBlock and TextBlock
Handles tokenization and numericalization automatically when `TextBlock` is passed to `DataBlock`

In [9]:
get_yelp = partial(get_text_files, folders=['train'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_yelp, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [10]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj typical college - y bar feel . $ 5 cover for … what ? i hate that i get charged cover to go into a bar . i live in xxmaj san xxmaj francisco , and there 's never a cover charge ( and the bars are way better ) . xxmaj and if there is a cover charge , it 's because the place is actually freaking great , and it 's usually worth it . \n\n","xxmaj typical college - y bar feel . $ 5 cover for … what ? i hate that i get charged cover to go into a bar . i live in xxmaj san xxmaj francisco , and there 's never a cover charge ( and the bars are way better ) . xxmaj and if there is a cover charge , it 's because the place is actually freaking great , and it 's usually worth it . \n\n xxmaj"
1,"do that . i would n't come here for dinner … \n\n xxmaj that being said , it 's a great location and the perfect spot for meeting up with friends for some game day brews . xxbos xxmaj man , this place is great ! xxmaj incredible atmosphere , friendly services , prices were great , and the food was great ! i had the jerk chicken wings and the flavor was fantastic . xxmaj although the portions were","that . i would n't come here for dinner … \n\n xxmaj that being said , it 's a great location and the perfect spot for meeting up with friends for some game day brews . xxbos xxmaj man , this place is great ! xxmaj incredible atmosphere , friendly services , prices were great , and the food was great ! i had the jerk chicken wings and the flavor was fantastic . xxmaj although the portions were nice"


We can see the dependent var is the same as the independent var offset by 1. 

$actodo -`show batch` auto denumericalizing text.  figure out how to see numbers.

Now that our data is ready, we can fine-tune the pretrained language model.

### Fine-Tuning the Language Model
Fine tune our lang model: need to create a learner that will learn to predict the next word of a movie review. 

To convert the integer word indices into activations that we can use for our neural network, we will use embeddings, just like we did for collaborative filtering and tabular modeling. Then we'll feed those embeddings into a *recurrent neural network* (RNN), using an architecture called *AWD-LSTM* (we will show you how to write such a model from scratch in <<chapter_nlp_dive>>). As we discussed earlier, the embeddings in the pretrained model are merged with random embeddings added for words that weren't in the pretraining vocabulary. This is handled automatically inside  
> fast.ai `language_model_learner`

In [12]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16() #to use less memore of GPU

The loss function used by default is cross-entropy loss, since we essentially have a classification problem (the different categories being the words in our vocab). The *perplexity* metric used here is often used in NLP for language models: it is the exponential of the loss (i.e., `torch.exp(cross_entropy)`). We  also include the accuracy metric, to see how many times our model is right when trying to predict the next word, since cross-entropy (as we've seen) is both hard to interpret, and tells us more about the model's confidence than its accuracy.

Let's go back to the process diagram from the beginning of this chapter. The first arrow has been completed for us and made available as a pretrained model in fastai, and we've just built the `DataLoaders` and `Learner` for the second stage. Now we're ready to fine-tune our language model!

<img alt="Diagram of the ULMFiT process" width="450" src="images/att_00027.png">

It takes quite a while to train each epoch, so we'll be saving the intermediate model results during the training process. Since `fine_tune` doesn't do that for us, we'll use `fit_one_cycle`. Just like `cnn_learner`, `language_model_learner` automatically calls `freeze` when using a pretrained model (which is the default), so this will only train the embeddings (the only part of the model that contains randomly initialized weights—i.e., embeddings for words that are in our IMDb vocab, but aren't in the pretrained model vocab):

In [ ]:
df100

In [37]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.911888,3.696903,0.298641,40.322224,1:03:56


This model takes a while to train, so it's a good opportunity to talk about saving intermediary results. 
### Saving and Loading Models
You can easily save the state of your model

In [38]:
learn.save('1epoch')

Path('/home/jupyter/.fastai/archive/yelp_reviews/models/1epoch.pth')

This will create a file in `learn.path/models/` named *1epoch.pth*. If you want to load your model in another machine after creating your `Learner` the same way, or resume training later, you can load the content of this file with:

In [13]:
learn = learn.load('1epoch')

Once the initial training has completed, we can continue fine-tuning the model after unfreezing:

In [14]:
learn.unfreeze()
learn.fit_one_cycle(4, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.512807,3.370524,0.339307,29.093767,3:00:41
1,3.398937,3.273032,0.351125,26.391235,1:08:30
2,3.330777,3.211477,0.358896,24.815720,1:07:59
3,3.266018,3.192218,0.361475,24.342356,1:07:08


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Once this is done, we save all of our model except the final layer that converts activations to probabilities of picking each token in our vocabulary. The model not including the final layer is called the *encoder*. We can save it with `save_encoder`

In [15]:
learn.save_encoder('finetuned')

### Text Generation
Try something different: using our model to generate random reviews. Since it's trained to guess what the next word of the sentence is, we can use the model to write new reviews

In [16]:
TEXT = "I liked this place because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [17]:
print("\n".join(preds))

i liked this place because i had the sandwich & fries combo . The sandwich was really good , the ingredients were fresh , the portion size was good . It was a little on the expensive side for such a small sandwich
i liked this place because of its location and the fact that i was in the area . The service was good and the food was pretty good . My boyfriend and i got the beef and mushroom pho and it was pretty


### Creating the Classifier DataLoaders
We're now moving from language model fine-tuning to classifier fine-tuning. To recap:  
- a language model predicts the next word of a document, so it doesn't need any external labels. 
- a classifier, however, predicts some external label—in the case of IMDb, it's the sentiment of a document.

This means that the structure of our `DataBlock` for NLP classification will look very familiar. It's actually nearly the same as we've seen for the many image classification datasets we've worked with.  Looking at the `DataBlock` definition, every piece is familiar from previous data blocks we've built, with two important exceptions:

- `TextBlock.from_folder` no longer has the `is_lm=True` parameter.
- We pass the `vocab` we created for the language model fine-tuning.

The reason that we pass the `vocab` of the language model is to make sure we use the same correspondence of token to index. Otherwise the embeddings we learned in our fine-tuned language model won't make any sense to this model, and the fine-tuning step won't be of any use.

By passing `is_lm=False` (or not passing `is_lm` at all, since it defaults to `False`) we tell `TextBlock` that we have regular labeled data, rather than using the next tokens as labels.

In [24]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train']), #, 'test'
    #splitter=GrandparentSplitter(valid_name='test')
    splitter=RandomSplitter(0.2)
).dataloaders(path, path=path, bs=128, seq_len=72)

In [25]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos a hands down must see attraction . xxmaj the first time i saw it in 2006 they played xxmaj my xxmaj heart xxmaj will xxmaj go xxmaj on by xxmaj celine xxmaj dion . xxmaj it was so beautiful i almost cried ! xxmaj the strip side offers the best views in my opinion . \n\n xxmaj the fountains are set in a 9 acre manmade lake . xxmaj they incorporate a network of pipes with more than 1,200 nozzles that make it possible to stage fountain displays coordinated with more than 4,500 lights . xxmaj it is estimated that the fountains cost $ 75 million to build . xxmaj the fountains were created by xxup wet , a design firm specializing in inventive fountains and architectural water features . \n\n xxmaj here are the fountain show times : \n\n xxmaj monday - xxmaj friday \n 3:00 xxup pm",positive
1,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,negative
2,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,negative


There is one challenge we have to deal with, however, which is to do with collating multiple documents into a mini-batch. Let's see with an example, by trying to create a mini-batch containing the first 10 documents. First we'll numericalize them

In [ ]:
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

The sorting and padding are automatically done by the data block API for us when using a TextBlock, with is_lm=False. (We don't have this same issue for language model data, since we concatenate all the documents together first, and then split them into equally sized sections.)

We can now create a model to classify our texts

In [29]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

The final step prior to training the classifier is to load the encoder from our fine-tuned language model. We use `load_encoder` instead of `load` because we only have pretrained weights available for the encoder; `load` by default raises an exception if an incomplete model is loaded

In [30]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning the Classifier
The last step is to train with discriminative learning rates and *gradual unfreezing*. In computer vision we often unfreeze the model all at once, but for NLP classifiers, we find that unfreezing a few layers at a time makes a real difference

In [31]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.194324,0.094416,0.964241,06:34


In just one epoch we get the same result as our training in chapter 1: not too bad! We can pass `-2` to `freeze_to` to freeze all except the last two parameter groups:

In [32]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.099464,0.075536,0.971723,07:57


Then we can unfreeze a bit more, and continue training

In [33]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.083847,0.064726,0.976286,11:42


And finally, the whole model!

In [34]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.068966,0.064197,0.976750,15:39
1,0.063224,0.062778,0.977804,15:43


We reached 94.3% accuracy, which was state-of-the-art performance just three years ago.

## Conclusion
In this chapter we explored the last application covered out of the box by the fastai library: text. We saw two types of models: language models that can generate texts, and a classifier that determines if a review is positive or negative. To build a state-of-the art classifier, we used a pretrained language model, fine-tuned it to the corpus of our task, then used its body (the encoder) with a new head to do the classification.

## Questionnaire

1. What is "self-supervised learning"?
1. What is a "language model"?
1. Why is a language model considered self-supervised?
1. What are self-supervised models usually used for?
1. Why do we fine-tune language models?
1. What are the three steps to create a state-of-the-art text classifier?
1. How do the 50,000 unlabeled movie reviews help us create a better text classifier for the IMDb dataset?
1. What are the three steps to prepare your data for a language model?
1. What is "tokenization"? Why do we need it?
1. Name three different approaches to tokenization.
1. What is `xxbos`?
1. List four rules that fastai applies to text during tokenization.
1. Why are repeated characters replaced with a token showing the number of repetitions and the character that's repeated?
1. What is "numericalization"?
1. Why might there be words that are replaced with the "unknown word" token?
1. With a batch size of 64, the first row of the tensor representing the first batch contains the first 64 tokens for the dataset. What does the second row of that tensor contain? What does the first row of the second batch contain? (Careful—students often get this one wrong! Be sure to check your answer on the book's website.)
1. Why do we need padding for text classification? Why don't we need it for language modeling?
1. What does an embedding matrix for NLP contain? What is its shape?
1. What is "perplexity"?
1. Why do we have to pass the vocabulary of the language model to the classifier data block?
1. What is "gradual unfreezing"?
1. Why is text generation always likely to be ahead of automatic identification of machine-generated texts?

1. See what you can learn about language models and disinformation. What are the best language models today? Take a look at some of their outputs. Do you find them convincing? How could a bad actor best use such a model to create conflict and uncertainty?
1. Given the limitation that models are unlikely to be able to consistently recognize machine-generated texts, what other approaches may be needed to handle large-scale disinformation campaigns that leverage deep learning?